In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from trading_middlewares.io.dataframe import ParquetDataFrame


In [3]:
import json
# dataframe = ParquetDataFrame(file_path="~/storage/data")
# 
# filter_dict = {'week_id': ["202441"]}
# data = dataframe.read(filter_dict=filter_dict)
# data.head()
# 
# unique_symbols = list(data['symbol'].unique())
# len(unique_symbols)
# 
# with open("../data/symbols.json", "w") as f:
#     json.dump(unique_symbols, f)
dataframe = ParquetDataFrame(file_path="~/storage/data")
with open("../data/symbols.json", "r") as f:
    unique_symbols = json.load(f)

In [4]:
symbol_ = unique_symbols[0]
data = dataframe.read(filter_dict={'symbol':[symbol_]})
data.head()

,date,open,high,low,close,volume,week_id,symbol
0,2014-10-23,20.0,20.0,20.0,20.0,0,201443,00631L.TW
1,2014-10-24,20.0,20.0,20.0,20.0,0,201443,00631L.TW
2,2014-10-27,20.0,20.0,20.0,20.0,0,201444,00631L.TW
3,2014-10-28,20.0,20.0,20.0,20.0,0,201444,00631L.TW
4,2014-10-29,20.0,20.0,20.0,20.0,0,201444,00631L.TW


In [90]:
from signal_research.indicators import VolumeAdjustedHighLowDelta, add_rolling_regression, add_bollinger_bands, add_average_true_range, add_donchian_channels
from signal_research.processing import generateWeeklyBars
from signal_research.plotting import plot_price_and_indicators, plot_histogram, plot_buy_sell_signals
from signal_research.evaluation import calculate_returns
from signal_research.signals import generate_regression_signal
data = data.dropna()
data_processed = generateWeeklyBars(data)
#data_processed = data.copy()
data_processed = data_processed.dropna()
data_processed = VolumeAdjustedHighLowDelta(data_processed, period=5)
data_processed = add_rolling_regression(data_processed, column="VolumeAdjustedHighLowDelta", window=4, confidence=0.95)


plot_price_and_indicators(data_processed, symbol_, indicators=[('VolumeAdjustedHighLowDelta', 'lines'), ('regression', 'lines'), ('regression', 'confidence')])

 

/home/aechraibi/code/signal-research/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

/home/aechraibi/code/signal-research/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2842: RuntimeWarning:

invalid value encountered in subtract



In [91]:
data_processed = add_bollinger_bands(data_processed, period=20)
data_processed = add_donchian_channels(data_processed, period=5)
plot_price_and_indicators(data_processed, symbol_, indicators=[('price', 'chart'), ('donchian', 'lines'), ('donchian', 'confidence')])

In [92]:
data_processed = generate_regression_signal(data_processed, indicator_name="VolumeAdjustedHighLowDelta")
data_processed = calculate_returns(data_processed, monthly=True)

In [93]:
data_processed = calculate_returns(data_processed, monthly=False)
data_processed.head()

,date,open,high,low,close,volume,VolumeAdjustedHighLowDelta,regression,regression_upper,regression_lower,...,donchian_lower,donchian,signal,1_month_return,3_month_return,6_month_return,9_month_return,1_year_return,5_year_return,return
0,2014-10-26,20.00,20.000000,20.000000,20.000000,0.0,-inf,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-11-02,20.00,20.200001,19.930000,20.200001,16734000.0,8.742705,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-11-09,20.18,20.650000,19.900000,20.129999,41948000.0,11.231912,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-11-16,20.17,20.870001,20.010000,20.320000,26519000.0,12.037644,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-11-23,20.51,20.920000,19.459999,19.480000,31841000.0,12.815247,NaN,NaN,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
plot_histogram(data_processed, 'return')

In [95]:
plot_buy_sell_signals(data_processed)